## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
# Step two: correlations

#importing all the libraries needed
import seaborn as sns
import matplotlib.pyplot as plt

# libraries to split and process data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Libraries for ML
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [2]:
promotion = train_data[train_data['Promotion']=='Yes']
non_promotion = train_data[train_data['Promotion']=='No']

In [3]:
Y = promotion.values[:,2]
X = promotion.values[:,3:]
Y=Y.astype('int')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.4, random_state=0)

In [5]:
y_test.sum()/len(y_test)

0.016169007435382982

### Dealing with Class imbalance
 

In [6]:
# define pipeline
steps = [('over', RandomOverSampler(sampling_strategy='minority')), ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)

In [7]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# sampling_strategy='minority' -> 
# minority class has half as many data points as the majority

# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, Y)

In [9]:
y_over.sum()/len(y_over)

0.5

### Build models
now that the data is balanced we will used ML models to predict whether someone is likely to purchase the product after receiving the promotion. Their results will be evaluated through Specificity and Sensitivity.

In [10]:
from sklearn.model_selection import train_test_split
>>> from sklearn import preprocessing
>>> X_train_val, X_test, y_train_val, y_test = train_test_split(
...     X_over, y_over, test_size=0.2, random_state=0)
>>> X_train, X_val, y_train, y_val = train_test_split(
...     X_train_val, y_train_val, test_size=0.2, random_state=0)
>>> scaler = preprocessing.StandardScaler().fit(X_train)
>>> X_train_transformed = scaler.transform(X_train)
>>> clf = LogisticRegression( ).fit(X_train_transformed, y_train)
>>> X_test_transformed = scaler.transform(X_test)
>>> clf.score(X_test_transformed, y_test)

from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test_transformed)


In [11]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

(tn, fp, fn, tp)

# Sensitivity  (also known as the “true positive” rate)
tp / (tp+fn)

# Specificity (also known as the “true negative” rate)
tn/(fp+tn)

0.37311984473556525

### Finding optimal model 
Using Sklearns pipelining method  


In [12]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [13]:
print(f"Decision Tree {mean(cross_val_score(DecisionTreeClassifier(), X_train_val,  y_train_val, scoring='f1_micro', cv=cv, n_jobs=-1))}")

print(f"Random Forrest {mean(cross_val_score(RandomForestClassifier( random_state=0), X_train_val,  y_train_val, scoring='f1_micro', cv=cv, n_jobs=-1))}")

print(f"Gaussian Naive Bayes {mean(cross_val_score(GaussianNB(), X_train_val,  y_train_val, scoring='f1_micro', cv=cv, n_jobs=-1))}")

Decision Tree 0.9886434289084153
Random Forrest 0.9957825625769345
Gaussian Naive Bayes 0.5993126022674825


In [14]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

# define pipeline
steps = [('over', RandomOverSampler(sampling_strategy='minority')),('scale',StandardScaler()), ('model',  SVC(gamma='auto'))]
pipeline = Pipeline(steps=steps)
print(f"SVM {mean(cross_val_score(pipeline, X_train_val,  y_train_val, scoring='f1_micro', cv=cv, n_jobs=-1))}")

In [15]:
clf = RandomForestClassifier( random_state=0)

In [16]:
clf.fit(X_train_val,  y_train_val)


RandomForestClassifier(random_state=0)

In [17]:
clf.score(X_test, y_test)

0.9962780645935887

In [18]:
y_pred = clf.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(tn, fp, fn, tp)

# Sensitivity  (also known as the “true positive” rate)
print(f'Sensitivity {tp / (tp+fn)}')

# Specificity (also known as the “true negative” rate)
print(f'Specificity {tn/(fp+tn)}')

8182 62 0 8414
Sensitivity 1.0
Specificity 0.992479378942261


In [21]:
y_test.sum()/len(y_test)

0.5051026533797575

In [79]:
Y = promotion['purchase'].values
X = promotion.iloc[:,3:].values
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# sampling_strategy='minority' -> 
# minority class has half as many data points as the majority

# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, Y)

X_train, X_test, y_train, y_test = train_test_split(
    X_over, y_over, test_size=0.2, random_state=0)
clfA = RandomForestClassifier( random_state=0).fit(X_train, y_train)

clfA.score(X_test, y_test)


y_pred = clfA.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(tn, fp, fn, tp)


# Sensitivity  (also known as the “true positive” rate)
print(f'Sensitivity {tp / (tp+fn)}')

# Specificity (also known as the “true negative” rate)
print(f'Specificity {tn/(fp+tn)}')

8174 70 0 8414
Sensitivity 1.0
Specificity 0.9915089762251335


### Model B design


In [77]:
non_promotion.iloc[:,3:].values

array([[ 2.        , 30.4435178 , -1.1650834 , ...,  1.        ,
         3.        ,  2.        ],
       [ 3.        , 32.1593501 , -0.6456167 , ...,  3.        ,
         2.        ,  2.        ],
       [ 2.        , 30.4316591 ,  0.13358341, ...,  1.        ,
         4.        ,  2.        ],
       ...,
       [ 1.        , 31.4920187 ,  1.08593908, ...,  3.        ,
         2.        ,  2.        ],
       [ 1.        , 37.7661061 ,  0.99936129, ...,  2.        ,
         1.        ,  2.        ],
       [ 1.        , 21.7208355 ,  1.08593908, ...,  2.        ,
         1.        ,  2.        ]])

In [78]:
Y = non_promotion['purchase'].values
X = non_promotion.iloc[:,3:].values
Y=Y.astype('int')
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# sampling_strategy='minority' -> 
# minority class has half as many data points as the majority

# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, Y)

X_train, X_test, y_train, y_test = train_test_split(
    X_over, y_over, test_size=0.2, random_state=0)
clfB = RandomForestClassifier( random_state=0).fit(X_train, y_train)

clfB.score(X_test, y_test)


y_pred = clfB.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(tn, fp, fn, tp)


# Sensitivity  (also known as the “true positive” rate)
print(f'Sensitivity {tp / (tp+fn)}')

# Specificity (also known as the “true negative” rate)
print(f'Specificity {tn/(fp+tn)}')

8259 16 0 8466
Sensitivity 1.0
Specificity 0.9980664652567975


## Combining models for final output

In [34]:
prediction_B

array([1, 0, 1, ..., 1, 0, 0])

In [35]:
d = {'prediction_A': prediction_A, 'prediction_B': prediction_B}

df = pd.DataFrame(data=d)
df

In [ ]:
df['output'] = df['output']

In [38]:
df['output'] = [1 if df['prediction_A'] == 1 and df['prediction_B'] ==0]

SyntaxError: invalid syntax (<ipython-input-38-d3e54b610a40>, line 1)

In [44]:
df.loc[df['prediction_A'] >0, 'bool_answer'] = 1
df.loc[df['prediction_B'] >0, 'bool_answer'] = 0
df.loc[df['prediction_A'] ==0, 'bool_answer'] = 0

In [45]:
df['bool_answer'].sum()

1

In [46]:
df

,prediction_A,prediction_B,output,bool_answer
0,1,1,No,0
1,0,0,No,0
2,1,1,No,0
3,1,1,No,0
4,1,1,No,0
...,...,...,...,...
16736,1,1,No,0
16737,0,0,No,0
16738,1,1,No,0
16739,0,0,No,0


In [33]:
prediction_A = clfA.predict(X_test)
prediction_B = clfB.predict(X_test)

In [61]:
train_data['purchase'].values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [52]:
Y_full[0]

4

In [80]:
X_full = train_data.iloc[:,3:].values
Y_full = train_data['purchase'].values
prediction_A = clfA.predict(X_full)
prediction_B = clfB.predict(X_full)

In [81]:
d = {'prediction_A': prediction_A, 'prediction_B': prediction_B}

df = pd.DataFrame(data=d)
df

,prediction_A,prediction_B
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
84529,0,0
84530,0,0
84531,0,0
84532,0,0


In [82]:
df.loc[df['prediction_A'] >0, 'bool_answer'] = 1
df.loc[df['prediction_B'] >0, 'bool_answer'] = 0
df.loc[df['prediction_A'] ==0, 'bool_answer'] = 0

In [83]:
df['bool_answer'].sum()

1098.0

In [84]:
df['purchase'] = Y_full

In [87]:
noB = df[df['prediction_B'] ==0]

In [88]:
noB

,prediction_A,prediction_B,bool_answer,purchase
0,0,0,0.0,0
1,0,0,0.0,0
2,0,0,0.0,0
3,0,0,0.0,0
4,0,0,0.0,0
...,...,...,...,...
84529,0,0,0.0,0
84530,0,0,0.0,0
84531,0,0,0.0,0
84532,0,0,0.0,0


In [89]:

tn, fp, fn, tp = confusion_matrix(noB['purchase'], noB['bool_answer']).ravel()

print(tn, fp, fn, tp)


83033 377 0 721


In [92]:
df

,prediction_A,prediction_B,bool_answer,purchase
0,0,0,0.0,0
1,0,0,0.0,0
2,0,0,0.0,0
3,0,0,0.0,0
4,0,0,0.0,0
...,...,...,...,...
84529,0,0,0.0,0
84530,0,0,0.0,0
84531,0,0,0.0,0
84532,0,0,0.0,0


In [ ]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    Y = promotion['purchase'].values
    X = promotion.iloc[:,3:].values
    # define oversampling strategy
    oversample = RandomOverSampler(sampling_strategy='minority')
    # sampling_strategy='minority' -> 
    # minority class has half as many data points as the majority

    # fit and apply the transform
    X_over, y_over = oversample.fit_resample(X, Y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_over, y_over, test_size=0.2, random_state=0)
    clfA = RandomForestClassifier( random_state=0).fit(X_train, y_train)

clfA.score(X_test, y_test)


y_pred = clfA.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(tn, fp, fn, tp)


# Sensitivity  (also known as the “true positive” rate)
print(f'Sensitivity {tp / (tp+fn)}')

# Specificity (also known as the “true negative” rate)
print(f'Specificity {tn/(fp+tn)}')
    
    
    return promotion